In [1]:
%env CUDA_LAUNCH_BLOCKING=1
%load_ext autoreload
%autoreload complete
# %aimport ...

import sys
import importlib
import warnings

importlib.invalidate_caches()

for mod_name in list(sys.modules):
    if not (str.startswith(mod_name, "robotodo") or str.startswith(mod_name, "tensorspecs")):
        continue
    try:
        importlib.reload(sys.modules[mod_name])
    except Exception as error:
        warnings.warn(f"{error}")

from robotodo.engines.isaac._kernel import Kernel

kernel = Kernel([
    "--/log/file=/tmp/todo-isaacsim.log",
    # "--/log/enableStandardStreamOutput=true",
    "--/app/enableStdoutOutput=true",
    # "--no-window",
    # TODO
    # "--/app/content/emptyStageOnStart=false",
    # "--/app/asyncRendering=true",
    # "--enable", "isaacsim.exp.full",
    "--/app/vulkan=",
], kit_path="isaacsim.exp.full.kit")
kernel.start_app_loop_soon()

env: CUDA_LAUNCH_BLOCKING=1
[Warning] [simulation_app] Interactive python shell detected but ISAAC_JUPYTER_KERNEL was not set. Problems with asyncio may occur
[Warning] [simulation_app] Please use Isaac Sim Python 3 kernel instead of the default Python 3 Kernel
Loading user config located at: '/home/sysadmin/lab/robotodo/.conda/lib/python3.11/site-packages/omni/data/Kit/Isaac-Sim Full/5.0/user.config.json'
[0.114s] [ext: omni.kit.async_engine-0.0.3] startup
[0.414s] [ext: omni.metrics.core-0.0.3] startup
[0.415s] [ext: omni.client.lib-1.1.0] startup
[0.428s] [ext: omni.blobkey-1.1.2] startup
[0.429s] [ext: omni.stats-1.0.1] startup
[0.430s] [ext: omni.datastore-0.0.0] startup
[0.435s] [ext: omni.client-1.3.0] startup
[0.490s] [ext: omni.ujitso.default-1.0.0] startup
[0.491s] [ext: omni.hsscclient-1.1.2] startup
[0.497s] [ext: omni.gpu_foundation.shadercache.vulkan-1.0.0] startup
[0.499s] [ext: omni.assets.plugins-0.0.0] startup
[0.500s] [ext: omni.gpu_foundation-0.0.0] startup
[0.506s]

2025-10-17T08:22:44Z s] [Error] [omni.gpu_foundation_factory.plugin] Start up failed. The default graphics plugin cannot be set!



|---------------------------------------------------------------------------------------------|
| Driver Version: 535.247.01    | Graphics API: Vulkan
|=============================================================================================|
| GPU | Name                             | Active | LDA | GPU Memory | Vendor-ID | LUID       |
|     |                                  |        |     |            | Device-ID | UUID       |
|     |                                  |        |     |            | Bus-ID    |            |
|---------------------------------------------------------------------------------------------|
| 0   | NVIDIA GeForce RTX 3060          | Yes: 0 |     | 12288   MB | 10de      | 0          |
|     |                                  |        |     |            | 2504      | 3157457f.. |
|     |                                  |        |     |            | 1         |            |
|===============================================================================

2025-10-17T08:22:58Z [14,021ms] [Error] [isaacsim.ros2.bridge.impl.extension] ROS2 Bridge startup failed


[14.892s] [ext: omni.kit.registry.nucleus-0.0.0] startup
[15.183s] app ready


2025-10-17T08:22:59Z [15,162ms] [Error] [carb.graphics-vulkan.plugin] Could not get NGX parameters block because NGX isn't enabled.
2025-10-17T08:22:59Z [15,162ms] [Error] [carb.graphics-vulkan.plugin] Failed to create NGX context.


2025-10-17T08:22:59Z [15,307ms] [Error] [rtx.optixdenoising73.plugin] Failed to create an Optix adaptor for device 0


In [ ]:
# TODO 
from robotodo.engines.isaac.builders import load_usd_scene

# TODO set up axis
scene = await load_usd_scene(
    "https://omniverse-content-production.s3-us-west-2.amazonaws.com/Assets/Isaac/5.0/Isaac/Environments/Simple_Room/simple_room.usd",
    # "https://omniverse-content-production.s3-us-west-2.amazonaws.com/Assets/Isaac/5.0/Isaac/Environments/Grid/default_environment.usd",
    # "https://omniverse-content-production.s3-us-west-2.amazonaws.com/Assets/Isaac/5.0/Isaac/Environments/Grid/gridroom_black.usd",
    _kernel=kernel,
)

In [ ]:
from robotodo.engines.isaac.scene import Scene
from robotodo.engines.isaac.builders import load_urdf, load_usd, build_camera
from robotodo.utils.pose import Pose


# scene = Scene(_kernel=kernel)


panda = await load_urdf(
    "./todo-curobo-sample-configs/franka/franka_description/franka_panda.urdf", 
    scene=scene, 
    path="/Panda",
    fix_root_link=True,
)


mug = await load_usd(
    "https://omniverse-content-production.s3-us-west-2.amazonaws.com/Assets/Isaac/5.0/Isaac/Props/Mugs/SM_Mug_A2.usd",
    scene=scene,
    path="/Mug",
)

mug.collision.enabled = True
mug.rigid_body.enabled = True
mug.pose = Pose(p=[.5, .5, .5])



[21.580s] Isaac Sim Full App is loaded.


In [4]:
mug.geometry

[[PolygonMesh(vertices=array([[-9.70949441e-17,  7.41638930e-02,  6.70092377e-02],
         [-1.05145170e-16,  6.79847321e-02,  7.39107497e-02],
         [-8.96764180e-17,  7.71494276e-02,  6.19627748e-02],
         ...,
         [ 2.65790933e-02, -2.00716205e-02,  2.28881831e-07],
         [ 2.83167451e-02, -1.75328918e-02,  2.41994852e-07],
         [ 2.98133271e-02, -1.48454377e-02,  2.46763224e-07]]), face_vertex_counts=array([3, 3, 3, ..., 3, 3, 3], dtype=int32), face_vertex_indices=array([ 131,  490,    0, ..., 2641, 2555, 2556], dtype=int32))]]

In [9]:
from curobo.cuda_robot_model.kinematics_parser import (
    KinematicsParser as CuroboKinematicsParser,
    LinkParams as CuroboLinkParams,
)

In [ ]:

class TODOCuroboKinematicsParser(CuroboKinematicsParser):
    def build_link_parent(self):
        self._parent_map
        raise NotImplementedError
    
    def get_link_parameters(self, link_name: str, base: bool = False) -> CuroboLinkParams:
        # CuroboLinkParams(
        #     link_name=link_name,
        #     joint_name=...,
        #     joint_type=...,
        #     fixed_transform=...,
        # )
        raise NotImplementedError

In [ ]:
import enum
from robotodo.utils.pose import Pose
# TODO
from tensorspecs import TensorLike

class BaseEntity:
    pose: Pose
    geometry: ...
    mass: ...
    mass_center_pose: Pose


class DOFKind(enum.Enum):
    NONE = -1
    ROTATION = 0
    TRANSLATION = 1

class BaseDOF:
    # TODO vecize
    kind: DOFKind
    limit: ...

class BaseJoint:
    # TODO
    # name: TensorLike["*", str]

    entity0: BaseEntity
    """The first entity attached to this joint."""
    entity1: BaseEntity
    """The second entity attached to this joint."""
    pose: Pose
    """The world pose of this joint."""
    local_pose0: Pose
    """The local pose of this joint in :attr:`entity0`."""
    local_pose1: Pose
    """The local pose of this joint in :attr:`entity1`."""
    # dofs: ... # TODO 

    rotation_limit: TensorLike["* xyz:3 minmax:2", "float"]
    translation_limit: TensorLike["* xyz:3 minmax:2", "float"]

"""
Joint(rotation_limit=[
    [0, 0], [0, 1], [0, 0]
])
"""

class BaseArticulation:
    joint: BaseJoint


In [ ]:
import lxml.etree

document = lxml.etree.parse("./todo-curobo-sample-configs/franka/franka_description/franka_panda.urdf")

root = document.getroot()
assert root.tag == "robot"
# TODO
root.attrib.get("name")

for element in root.iterchildren():
    match element.tag:
        case "link":
            element.attrib.get("name")
            
            element.iterfind("inertial")
            # TODO visual
            # TODO collision

        case "joint":
            element.attrib["name"]
            element.attrib["type"]

            for parent in element.iterfind("parent"):
                parent.attrib["link"]
            
            for child in element.iterfind("child"):
                child.attrib["link"]

            for origin in element.iterfind("origin"):
                origin.attrib.get("xyz", [0, 0, 0])
                origin.attrib.get("rpy", [0, 0, 0])


            
        case _:
            pass
    print(element.tag, element.attrib)

link {'name': 'base_link'}
joint {'name': 'panda_fixed', 'type': 'fixed'}
link {'name': 'panda_link0'}
link {'name': 'panda_link1'}
joint {'name': 'panda_joint1', 'type': 'revolute'}
link {'name': 'panda_link2'}
joint {'name': 'panda_joint2', 'type': 'revolute'}
link {'name': 'panda_link3'}
joint {'name': 'panda_joint3', 'type': 'revolute'}
link {'name': 'panda_link4'}
joint {'name': 'panda_joint4', 'type': 'revolute'}
link {'name': 'panda_link5'}
joint {'name': 'panda_joint5', 'type': 'revolute'}
link {'name': 'panda_link6'}
joint {'name': 'panda_joint6', 'type': 'revolute'}
link {'name': 'panda_link7'}
joint {'name': 'panda_joint7', 'type': 'revolute'}
link {'name': 'panda_link8'}
joint {'name': 'panda_joint8', 'type': 'fixed'}
joint {'name': 'panda_hand_joint', 'type': 'fixed'}
link {'name': 'panda_hand'}
link {'name': 'panda_leftfinger'}
link {'name': 'panda_rightfinger'}
joint {'name': 'panda_finger_joint1', 'type': 'prismatic'}
joint {'name': 'panda_finger_joint2', 'type': 'prism

In [11]:
root.attrib

{'name': 'panda'}

'robot'